## Chain of Thought Reasoning

Chain of Thought (CoT) is a prompting technique that asks language models to show their step-by-step thinking before giving a final answer. This approach:

- **Improves accuracy** on complex reasoning tasks
- **Provides transparency** into the model's thinking process
- **Works best for**: math problems, logic puzzles, and multi-step reasoning

However, CoT increases token usage and response time, so it's less suitable for simple tasks or latency-sensitive applications.

In this notebook, we'll explore how to implement CoT prompting and see its impact on response quality.

## Basic Chain of Thought

Let's start with a simple example to see the difference between regular prompting and chain of thought prompting:

In [ ]:
import boto3
import json

import boto3
import json

REGION = 'us-west-2'

# Initialize the Bedrock client
session = boto3.Session()
bedrock = session.client(service_name='bedrock-runtime', region_name=REGION)

print("✅ Setup complete!")

## Helper Functions and Templates

The code below defines reusable components for our Chain of Thought examples:
- A system prompt that instructs Claude to show its reasoning process
- A prompt template for structuring questions
- A function to call Claude via Bedrock
- A utility to extract content from XML tags using regex

In [ ]:

import re
from typing import Dict, Any, Optional

# Define a system prompt that encourages Chain of Thought.
SYSTEM_PROMPT = """
You are a helpful assistant that can solve math word problems. 
<instruction>
You think through problems step by step and provide answers.
You place your thinking process in <thinking> tags and your final answer in <response> tags.
</instruction>
"""

PROMPT_TEMPLATE = """
Using the context provided, solve the question.

<question>{question}</question>

Think through the problem step by step and place your reasoning in <thinking> tags. When you have a final answer, place it in <response> tags.
"""

MODEL_ID: str = "anthropic.claude-3-5-haiku-20241022-v1:0"

# Helper function to call bedrock
def call_bedrock(prompt: str) -> str:

    # Create the message in Bedrock's required format
    user_message: Dict[str, Any] = { "role": "user","content": [{ "text": prompt}] }

    # Configure model parameters
    inference_config: Dict[str, Any] = {
        "temperature": .4,
        "maxTokens": 1000
    }

    # Send request to Claude Haiku 3.5 via Bedrock
    response: Dict[str, Any] = bedrock.converse(
        modelId=MODEL_ID,  # Using Sonnet 3.5 
        messages=[user_message],
        system=[{"text": SYSTEM_PROMPT}],
        inferenceConfig=inference_config
    )

    # Get the model's text response
    return response['output']['message']['content'][0]['text']

# Define a function to extract content from XML tags
def extract_tag_content(text: str, tag_name: str) -> Optional[str]:
    """
    Extract content between specified XML tags from text.
    """
    pattern = f'<{tag_name}>(.*?)</{tag_name}>'
    match = re.search(pattern, text, re.DOTALL)
    return match.group(1).strip() if match else None

In [ ]:
from typing import Optional

# Define the math problem as a variable to make it reusable
math_question = "If John has 5 apples and gives 2 to Mary, who then gives 1 to Tom, how many apples does Mary have?"

# For reusing prompts, it's useful to have a dictionary of inputs.
inputs = {
    'question': math_question,
}

# Format the prompt by filling in the variable.
prompt: str = PROMPT_TEMPLATE.format(**inputs)

# Call the model.
model_text: str = call_bedrock(prompt)

# Extract the thinking and response sections
thinking: Optional[str] = extract_tag_content(model_text, "thinking")
final_answer: Optional[str] = extract_tag_content(model_text, "response")

# Print the extracted sections
print("\n===== CHAIN OF THOUGHT REASONING =====")
print(thinking if thinking else "No thinking section found")

print("\n===== FINAL ANSWER =====")
print(final_answer if final_answer else "No response section found")

Notice how in the Chain of Thought example, we get to see the model's reasoning process! This is incredibly valuable when:
1. Debugging incorrect responses
2. Verifying the model's logic
3. Building trust in the model's outputs

## More Complex Example

Let's try a more complex problem that really shows the power of chain of thought reasoning:

In [ ]:
math_question = "A store has a 20% off sale. Alice buys a shirt for $40 and pants for $60. The store has a policy that the discount applies to the more expensive item first. What is her total after the discount?"

# For reusing prompts, it's useful to have a dictionary of inputs.
inputs = {
    'question': math_question,
}

# Format the prompt by filling in the variable.
prompt: str = PROMPT_TEMPLATE.format(**inputs)

# Call the model.
model_text: str = call_bedrock(prompt)

# Extract the thinking and response sections
thinking: Optional[str] = extract_tag_content(model_text, "thinking")
final_answer: Optional[str] = extract_tag_content(model_text, "response")

# Print the extracted sections
print("\n===== CHAIN OF THOUGHT REASONING =====")
print(thinking if thinking else "No thinking section found")

print("\n===== FINAL ANSWER =====")
print(final_answer if final_answer else "No response section found")


## Exercise

Now it's your turn! Try writing a chain of thought prompt for this problem:

"In a game, players score points based on collecting gems. Red gems are worth 5 points, blue gems are worth 3 points, and green gems are worth 2 points. If a player has 4 red gems, 3 blue gems, and 6 green gems, what is their total score?"

Remember to encourage the model to show its work!